<a href="https://colab.research.google.com/github/nsubbaian/FrequentistML/blob/master/Project1/FMLAssignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Frequentist Machine Learning Assigment 1: Linear Regression** 

In [174]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


# Selected Dataset: http://archive.ics.uci.edu/ml/datasets/Forest+Fires
# Dataset Description: https://archive.ics.uci.edu/ml/machine-learning-databases/forest-fires/forestfires.names
# Chosen so it's good for regression (with numerical, not categorical features, and the target is a continuous number)

dataset = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/forest-fires/forestfires.csv')

# removed categorical feautures
dataset = dataset.drop(columns = {'month', 'day'})

# Shuffle and then split training data into 80% train, 10% validation and 10% test
training, validation, test = np.split(dataset.sample(frac=1), [int(.8*len(dataset)), int(.9*len(dataset))])
print("Length of dataset:", len(dataset))
print( "Length of training:", len(training))
print("Length of validation:", len(validation))
print("Length of test:", len(test))

training

Length of dataset: 517
Length of training: 413
Length of validation: 52
Length of test: 52


,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
10,7,5,92.5,88.0,698.6,7.1,17.8,51,7.2,0.0,0.00
410,6,3,84.1,7.3,52.8,2.7,14.7,42,2.7,0.0,0.00
129,2,5,92.6,46.5,691.8,8.8,15.4,35,0.9,0.0,0.00
333,1,4,92.1,87.7,721.1,9.5,18.1,54,3.1,0.0,2.13
366,4,5,91.1,132.3,812.1,12.5,15.9,38,5.4,0.0,1.75
...,...,...,...,...,...,...,...,...,...,...,...
85,1,2,92.9,137.0,706.4,9.2,21.5,15,0.9,0.0,0.00
120,3,4,91.5,145.4,608.2,10.7,10.3,74,2.2,0.0,0.00
337,6,3,91.6,108.4,764.0,6.2,23.0,34,2.2,0.0,56.04
148,2,2,94.8,108.3,647.1,17.0,17.4,43,6.7,0.0,1.07


### A. Plain old linear regression, with no regularization

In [176]:
linTrain_X, linTrain_Y = training.iloc[:, :10], np.log(training.iloc[:, 10]+.001)
linTest_X, linTest_Y = test.iloc[:, :10], np.log(test.iloc[:,10]+.01)

#  Using eq 3.6 to find the betas
beta_hat = np.linalg.inv(linTrain_X.T.dot(linTrain_X)).dot(linTrain_X.T).dot(linTrain_Y)

y_hat = linTest_X.dot(beta_hat)

# mean squared error on the test dataset
MSE = np.square(np.subtract(linTest_Y,y_hat)).mean() 
print(MSE)

# display(pd.concat([linTest_Y, y_hat, np.subtract(linTest_Y,y_hat)], axis=1))

15.231355411266092


### B. Ridge Regression 

In [ ]:
RidgeTrain_X, RidgeTrain_Y = training.iloc[:, :10], np.log(training.iloc[:, 10]+.001)
RidgeVal_X, RidgeVal_Y = validation.iloc[:, :10], np.log(validation.iloc[:, 10]+.001)
RidgeTest_X, RidgeTest_Y = test.iloc[:, :10], np.log(test.iloc[:,10]+.01)

identity = 
lam = .1

# using eq 3.44 to find the betas
betaRidge_hat = np.linalg.inv((RidgeTrain_X.T.dot(RidgeTrain_X) + lam.dot(identity))).dot(RidgeTrain_X.T).dot(RidgeTrain_Y)

# select the optimal value of Lambda by cross-validation using the validation dataset

# Report the mean squared error on the test dataset, using the best lambda you found on the validation set. DO NOT USE THE TEST DATASET TO CHOOSE LAMBDA.

### C. Lasso Regression

In [ ]:
linTrain_X, linTrain_Y = training.iloc[:, :10], np.log(training.iloc[:, 10]+.001)
linTest_X, linTest_Y = test.iloc[:, :10], np.log(test.iloc[:,10]+.01)

# using sci-kit learn or MATLAB

# select the optimal value of lambda as in part B.

# Display a Lasso plot (there are built in functions for Lasso plot in sci-kit/MATLAB). Which features did the Lasso select for you to include in your model? Do these features make sense?


In [ ]:
# Compute the MSE on the training dataset and the test dataset for all methods and comment on the results.  


# Feeling brave? Do Lasso and Ridge plots(like figures 3.8 and 3.10).